In [13]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import cv2
from sklearn.model_selection import train_test_split
import os
import pickle
from PIL import Image
from resizeimage import resizeimage

In [2]:
device = torch.device('cpu')  # sad reax only

In [3]:
x = []  # image data, start as list
y = []  # output class data, start as list

for folder in os.listdir("./pictures/"):
    for file in os.listdir("./pictures/" + folder + "/"):
        im = cv2.imread("./pictures/" + folder + "/" + file)
        im = np.reshape(im, (3, 256, 144))
        x.append(im)
        out = np.zeros(9)
        out[int(folder)] = 1
        # out = int(folder)
        y.append(out)

x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.002, random_state=0)

In [4]:
class TrainDataset(Dataset):
    def __init__(self):
        self.len = x_train.shape[0]
        self.x_data = torch.from_numpy(x_train).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_train).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [5]:
class TestDataset(Dataset):
    def __init__(self):
        self.len = x_test.shape[0]
        self.x_data = torch.from_numpy(x_test).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_test).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [6]:
class Net(nn.Module):
    def __init__(self, num_classes=9):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5, 1)
        self.conv2 = nn.Conv2d(5, 10, 5, 1)
        self.fc1 = nn.Linear(20130, 10)
        self.fc2 = nn.Linear(10, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 20130)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
learning_rate = 0.002
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #print(data)
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data))
    if epoch == 10:
        pkl_filename = "pickle_model.pkl"
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)

In [9]:
if __name__ == '__main__':
    train_dataset = TrainDataset()
    train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

    test_dataset = TestDataset()
    test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

    """for epoch in range(5):
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            print(epoch, i, "inputs", inputs.data, "labels", labels.data)"""

    for epoch in range(1, 11):
        train(epoch, train_loader)

        # test(test_loader)

Train Epoch: 1 [0/4115 (0%)]	Loss: 12.032330
Train Epoch: 1 [10/4115 (0%)]	Loss: 2.423890
Train Epoch: 1 [20/4115 (0%)]	Loss: 2.348383
Train Epoch: 1 [30/4115 (1%)]	Loss: 1.440912
Train Epoch: 1 [40/4115 (1%)]	Loss: 2.125552
Train Epoch: 1 [50/4115 (1%)]	Loss: 1.955941
Train Epoch: 1 [60/4115 (1%)]	Loss: 2.121418
Train Epoch: 1 [70/4115 (2%)]	Loss: 2.330517
Train Epoch: 1 [80/4115 (2%)]	Loss: 2.243857
Train Epoch: 1 [90/4115 (2%)]	Loss: 2.249680
Train Epoch: 1 [100/4115 (2%)]	Loss: 2.031377
Train Epoch: 1 [110/4115 (3%)]	Loss: 2.124444
Train Epoch: 1 [120/4115 (3%)]	Loss: 2.310615
Train Epoch: 1 [130/4115 (3%)]	Loss: 2.122949
Train Epoch: 1 [140/4115 (3%)]	Loss: 2.152632
Train Epoch: 1 [150/4115 (4%)]	Loss: 2.123690
Train Epoch: 1 [160/4115 (4%)]	Loss: 1.950477
Train Epoch: 1 [170/4115 (4%)]	Loss: 2.587737
Train Epoch: 1 [180/4115 (4%)]	Loss: 2.280167
Train Epoch: 1 [190/4115 (5%)]	Loss: 2.208837
Train Epoch: 1 [200/4115 (5%)]	Loss: 2.580478
Train Epoch: 1 [210/4115 (5%)]	Loss: 2.56778

Train Epoch: 1 [1730/4115 (42%)]	Loss: 2.290094
Train Epoch: 1 [1740/4115 (42%)]	Loss: 2.262205
Train Epoch: 1 [1750/4115 (43%)]	Loss: 2.052188
Train Epoch: 1 [1760/4115 (43%)]	Loss: 2.292574
Train Epoch: 1 [1770/4115 (43%)]	Loss: 2.320856
Train Epoch: 1 [1780/4115 (43%)]	Loss: 2.271705
Train Epoch: 1 [1790/4115 (43%)]	Loss: 2.042396
Train Epoch: 1 [1800/4115 (44%)]	Loss: 2.281697
Train Epoch: 1 [1810/4115 (44%)]	Loss: 2.269699
Train Epoch: 1 [1820/4115 (44%)]	Loss: 2.299221
Train Epoch: 1 [1830/4115 (44%)]	Loss: 2.286281
Train Epoch: 1 [1840/4115 (45%)]	Loss: 2.267732
Train Epoch: 1 [1850/4115 (45%)]	Loss: 2.268619
Train Epoch: 1 [1860/4115 (45%)]	Loss: 2.093364
Train Epoch: 1 [1870/4115 (45%)]	Loss: 2.277665
Train Epoch: 1 [1880/4115 (46%)]	Loss: 2.026359
Train Epoch: 1 [1890/4115 (46%)]	Loss: 2.353735
Train Epoch: 1 [1900/4115 (46%)]	Loss: 2.024701
Train Epoch: 1 [1910/4115 (46%)]	Loss: 2.062965
Train Epoch: 1 [1920/4115 (47%)]	Loss: 2.107227
Train Epoch: 1 [1930/4115 (47%)]	Loss: 2

Train Epoch: 1 [3430/4115 (83%)]	Loss: 2.479950
Train Epoch: 1 [3440/4115 (84%)]	Loss: 2.136353
Train Epoch: 1 [3450/4115 (84%)]	Loss: 2.063103
Train Epoch: 1 [3460/4115 (84%)]	Loss: 2.348492
Train Epoch: 1 [3470/4115 (84%)]	Loss: 2.119132
Train Epoch: 1 [3480/4115 (85%)]	Loss: 2.310679
Train Epoch: 1 [3490/4115 (85%)]	Loss: 2.126731
Train Epoch: 1 [3500/4115 (85%)]	Loss: 2.003891
Train Epoch: 1 [3510/4115 (85%)]	Loss: 1.998934
Train Epoch: 1 [3520/4115 (86%)]	Loss: 2.310233
Train Epoch: 1 [3530/4115 (86%)]	Loss: 2.487571
Train Epoch: 1 [3540/4115 (86%)]	Loss: 2.126868
Train Epoch: 1 [3550/4115 (86%)]	Loss: 2.308036
Train Epoch: 1 [3560/4115 (87%)]	Loss: 2.111781
Train Epoch: 1 [3570/4115 (87%)]	Loss: 2.005717
Train Epoch: 1 [3580/4115 (87%)]	Loss: 2.074328
Train Epoch: 1 [3590/4115 (87%)]	Loss: 2.362685
Train Epoch: 1 [3600/4115 (87%)]	Loss: 2.363495
Train Epoch: 1 [3610/4115 (88%)]	Loss: 2.363577
Train Epoch: 1 [3620/4115 (88%)]	Loss: 2.115494
Train Epoch: 1 [3630/4115 (88%)]	Loss: 2

Train Epoch: 2 [1040/4115 (25%)]	Loss: 2.065409
Train Epoch: 2 [1050/4115 (26%)]	Loss: 2.300037
Train Epoch: 2 [1060/4115 (26%)]	Loss: 2.075238
Train Epoch: 2 [1070/4115 (26%)]	Loss: 1.993991
Train Epoch: 2 [1080/4115 (26%)]	Loss: 2.076047
Train Epoch: 2 [1090/4115 (26%)]	Loss: 2.311863
Train Epoch: 2 [1100/4115 (27%)]	Loss: 2.305408
Train Epoch: 2 [1110/4115 (27%)]	Loss: 2.075871
Train Epoch: 2 [1120/4115 (27%)]	Loss: 2.312238
Train Epoch: 2 [1130/4115 (27%)]	Loss: 2.441918
Train Epoch: 2 [1140/4115 (28%)]	Loss: 2.069298
Train Epoch: 2 [1150/4115 (28%)]	Loss: 2.444960
Train Epoch: 2 [1160/4115 (28%)]	Loss: 2.086344
Train Epoch: 2 [1170/4115 (28%)]	Loss: 2.079627
Train Epoch: 2 [1180/4115 (29%)]	Loss: 2.080361
Train Epoch: 2 [1190/4115 (29%)]	Loss: 2.080831
Train Epoch: 2 [1200/4115 (29%)]	Loss: 1.994007
Train Epoch: 2 [1210/4115 (29%)]	Loss: 2.079407
Train Epoch: 2 [1220/4115 (30%)]	Loss: 2.069852
Train Epoch: 2 [1230/4115 (30%)]	Loss: 2.308445
Train Epoch: 2 [1240/4115 (30%)]	Loss: 2

Train Epoch: 2 [2740/4115 (67%)]	Loss: 2.105335
Train Epoch: 2 [2750/4115 (67%)]	Loss: 2.099240
Train Epoch: 2 [2760/4115 (67%)]	Loss: 2.390996
Train Epoch: 2 [2770/4115 (67%)]	Loss: 2.494193
Train Epoch: 2 [2780/4115 (68%)]	Loss: 2.031315
Train Epoch: 2 [2790/4115 (68%)]	Loss: 2.477042
Train Epoch: 2 [2800/4115 (68%)]	Loss: 2.010941
Train Epoch: 2 [2810/4115 (68%)]	Loss: 2.086703
Train Epoch: 2 [2820/4115 (69%)]	Loss: 2.008907
Train Epoch: 2 [2830/4115 (69%)]	Loss: 2.009620
Train Epoch: 2 [2840/4115 (69%)]	Loss: 2.078125
Train Epoch: 2 [2850/4115 (69%)]	Loss: 2.281397
Train Epoch: 2 [2860/4115 (70%)]	Loss: 2.106638
Train Epoch: 2 [2870/4115 (70%)]	Loss: 1.980746
Train Epoch: 2 [2880/4115 (70%)]	Loss: 1.969002
Train Epoch: 2 [2890/4115 (70%)]	Loss: 2.093405
Train Epoch: 2 [2900/4115 (70%)]	Loss: 2.470058
Train Epoch: 2 [2910/4115 (71%)]	Loss: 2.045354
Train Epoch: 2 [2920/4115 (71%)]	Loss: 2.113069
Train Epoch: 2 [2930/4115 (71%)]	Loss: 2.084261
Train Epoch: 2 [2940/4115 (71%)]	Loss: 1

...

Train Epoch: 9 [1340/4115 (33%)]	Loss: 2.044570
Train Epoch: 9 [1350/4115 (33%)]	Loss: 2.017454
Train Epoch: 9 [1360/4115 (33%)]	Loss: 2.086060
Train Epoch: 9 [1370/4115 (33%)]	Loss: 2.037431
Train Epoch: 9 [1380/4115 (34%)]	Loss: 2.032417
Train Epoch: 9 [1390/4115 (34%)]	Loss: 2.086016
Train Epoch: 9 [1400/4115 (34%)]	Loss: 2.101512
Train Epoch: 9 [1410/4115 (34%)]	Loss: 2.086661
Train Epoch: 9 [1420/4115 (35%)]	Loss: 2.590780
Train Epoch: 9 [1430/4115 (35%)]	Loss: 2.095150
Train Epoch: 9 [1440/4115 (35%)]	Loss: 2.090056
Train Epoch: 9 [1450/4115 (35%)]	Loss: 2.084670
Train Epoch: 9 [1460/4115 (35%)]	Loss: 2.035307
Train Epoch: 9 [1470/4115 (36%)]	Loss: 2.425529
Train Epoch: 9 [1480/4115 (36%)]	Loss: 2.081184
Train Epoch: 9 [1490/4115 (36%)]	Loss: 2.363381
Train Epoch: 9 [1500/4115 (36%)]	Loss: 2.255067
Train Epoch: 9 [1510/4115 (37%)]	Loss: 2.087120
Train Epoch: 9 [1520/4115 (37%)]	Loss: 2.057198
Train Epoch: 9 [1530/4115 (37%)]	Loss: 2.092283
Train Epoch: 9 [1540/4115 (37%)]	Loss: 2

Train Epoch: 9 [3040/4115 (74%)]	Loss: 2.078255
Train Epoch: 9 [3050/4115 (74%)]	Loss: 2.055353
Train Epoch: 9 [3060/4115 (74%)]	Loss: 1.981318
Train Epoch: 9 [3070/4115 (75%)]	Loss: 2.352907
Train Epoch: 9 [3080/4115 (75%)]	Loss: 2.093710
Train Epoch: 9 [3090/4115 (75%)]	Loss: 1.974895
Train Epoch: 9 [3100/4115 (75%)]	Loss: 2.533745
Train Epoch: 9 [3110/4115 (76%)]	Loss: 2.087091
Train Epoch: 9 [3120/4115 (76%)]	Loss: 2.308964
Train Epoch: 9 [3130/4115 (76%)]	Loss: 2.343238
Train Epoch: 9 [3140/4115 (76%)]	Loss: 2.073469
Train Epoch: 9 [3150/4115 (77%)]	Loss: 2.345438
Train Epoch: 9 [3160/4115 (77%)]	Loss: 2.309222
Train Epoch: 9 [3170/4115 (77%)]	Loss: 2.090539
Train Epoch: 9 [3180/4115 (77%)]	Loss: 2.297740
Train Epoch: 9 [3190/4115 (78%)]	Loss: 2.074312
Train Epoch: 9 [3200/4115 (78%)]	Loss: 1.955256
Train Epoch: 9 [3210/4115 (78%)]	Loss: 2.293616
Train Epoch: 9 [3220/4115 (78%)]	Loss: 2.097680
Train Epoch: 9 [3230/4115 (78%)]	Loss: 2.552831
Train Epoch: 9 [3240/4115 (79%)]	Loss: 2

Train Epoch: 10 [630/4115 (15%)]	Loss: 2.302984
Train Epoch: 10 [640/4115 (16%)]	Loss: 2.114754
Train Epoch: 10 [650/4115 (16%)]	Loss: 2.262607
Train Epoch: 10 [660/4115 (16%)]	Loss: 1.998943
Train Epoch: 10 [670/4115 (16%)]	Loss: 1.996294
Train Epoch: 10 [680/4115 (17%)]	Loss: 2.270008
Train Epoch: 10 [690/4115 (17%)]	Loss: 2.571528
Train Epoch: 10 [700/4115 (17%)]	Loss: 2.299560
Train Epoch: 10 [710/4115 (17%)]	Loss: 2.048983
Train Epoch: 10 [720/4115 (17%)]	Loss: 2.065384
Train Epoch: 10 [730/4115 (18%)]	Loss: 2.263433
Train Epoch: 10 [740/4115 (18%)]	Loss: 2.061845
Train Epoch: 10 [750/4115 (18%)]	Loss: 2.110258
Train Epoch: 10 [760/4115 (18%)]	Loss: 2.575808
Train Epoch: 10 [770/4115 (19%)]	Loss: 2.111618
Train Epoch: 10 [780/4115 (19%)]	Loss: 2.044920
Train Epoch: 10 [790/4115 (19%)]	Loss: 2.043585
Train Epoch: 10 [800/4115 (19%)]	Loss: 1.980328
Train Epoch: 10 [810/4115 (20%)]	Loss: 2.257267
Train Epoch: 10 [820/4115 (20%)]	Loss: 2.075957
Train Epoch: 10 [830/4115 (20%)]	Loss: 2

Train Epoch: 10 [2300/4115 (56%)]	Loss: 2.591728
Train Epoch: 10 [2310/4115 (56%)]	Loss: 2.056472
Train Epoch: 10 [2320/4115 (56%)]	Loss: 2.063327
Train Epoch: 10 [2330/4115 (57%)]	Loss: 2.107415
Train Epoch: 10 [2340/4115 (57%)]	Loss: 2.059190
Train Epoch: 10 [2350/4115 (57%)]	Loss: 2.307129
Train Epoch: 10 [2360/4115 (57%)]	Loss: 2.008175
Train Epoch: 10 [2370/4115 (58%)]	Loss: 2.306456
Train Epoch: 10 [2380/4115 (58%)]	Loss: 2.033367
Train Epoch: 10 [2390/4115 (58%)]	Loss: 2.114535
Train Epoch: 10 [2400/4115 (58%)]	Loss: 2.034848
Train Epoch: 10 [2410/4115 (59%)]	Loss: 2.110263
Train Epoch: 10 [2420/4115 (59%)]	Loss: 2.008429
Train Epoch: 10 [2430/4115 (59%)]	Loss: 2.055207
Train Epoch: 10 [2440/4115 (59%)]	Loss: 2.253177
Train Epoch: 10 [2450/4115 (60%)]	Loss: 2.000148
Train Epoch: 10 [2460/4115 (60%)]	Loss: 2.039431
Train Epoch: 10 [2470/4115 (60%)]	Loss: 2.240211
Train Epoch: 10 [2480/4115 (60%)]	Loss: 1.995204
Train Epoch: 10 [2490/4115 (61%)]	Loss: 2.449993
Train Epoch: 10 [250

Train Epoch: 10 [3970/4115 (96%)]	Loss: 2.310589
Train Epoch: 10 [3980/4115 (97%)]	Loss: 2.396489
Train Epoch: 10 [3990/4115 (97%)]	Loss: 2.452008
Train Epoch: 10 [4000/4115 (97%)]	Loss: 1.963604
Train Epoch: 10 [4010/4115 (97%)]	Loss: 2.039925
Train Epoch: 10 [4020/4115 (98%)]	Loss: 2.302733
Train Epoch: 10 [4030/4115 (98%)]	Loss: 2.154912
Train Epoch: 10 [4040/4115 (98%)]	Loss: 2.089895
Train Epoch: 10 [4050/4115 (98%)]	Loss: 2.481104
Train Epoch: 10 [4060/4115 (99%)]	Loss: 2.406688
Train Epoch: 10 [4070/4115 (99%)]	Loss: 2.075281
Train Epoch: 10 [4080/4115 (99%)]	Loss: 2.304552
Train Epoch: 10 [4090/4115 (99%)]	Loss: 2.303665
Train Epoch: 10 [4100/4115 (100%)]	Loss: 2.038972
Train Epoch: 10 [4110/4115 (100%)]	Loss: 1.959677


In [24]:
while True:
    for file in os.listdir("../../../../Downloads/"):
        l = []
        if 'ss_file' in file:
            with open("../../../../Downloads/" + file, 'r+b') as f:
                with Image.open(f) as image:
                    resized_image = resizeimage.resize_cover(image, [256, 144])
                    resized_image.save("../../../../Downloads/" + file, image.format)
            im = cv2.imread("../../../../Downloads/" + file)
            im = np.reshape(im, (1, 3, 256, 144))
            x = np.array(im)
            x = torch.from_numpy(x).type(torch.FloatTensor)
            output = model(x)
            try:
                os.remove("file.txt")
            except:
                pass
            output = output.data.numpy().tolist()
            output = output[0]
            output = output.index(max(output))
            %store output >> file.txt
            os.remove("../../../../Downloads/" + file)

Writing 'output' (int) to file 'file.txt'.


KeyboardInterrupt: 